In [1]:
# loading package
import pandas as pd
import numpy as np

# read in data

In [207]:
# Read in data into dataframes 
test = pd.read_csv('processed_data/test.csv')
X = pd.read_csv('processed_data/X.csv')
X_test = pd.read_csv('processed_data/X_test.csv')
y = pd.read_csv('processed_data/y.csv')
y_test = pd.read_csv('processed_data/y_test.csv')

# Display sizes of data
print('Training Feature Size: ', X.shape)
print('Testing Feature Size:  ', X_test.shape)
print('Training Labels Size:  ', y.shape)
print('Testing Labels Size:   ', y_test.shape)
print('Real Testing Size:     ', test.shape)

Training Feature Size:  (45644, 811)
Testing Feature Size:   (11412, 811)
Training Labels Size:   (45644, 1)
Testing Labels Size:    (11412, 1)
Real Testing Size:      (14266, 811)


# random forest

In [3]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X, y.values.ravel())

RandomForestClassifier()

In [4]:
from sklearn.metrics import accuracy_score

model_pred = rfc.predict(X_test)
accuracy_score(y_test, model_pred)

0.8397301086575535

# random forest 2

In [193]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [194]:
pipeline = make_pipeline(MinMaxScaler(),
                         RandomForestClassifier())

print(pipeline.get_params())

{'memory': None, 'steps': [('minmaxscaler', MinMaxScaler()), ('randomforestclassifier', RandomForestClassifier())], 'verbose': False, 'minmaxscaler': MinMaxScaler(), 'randomforestclassifier': RandomForestClassifier(), 'minmaxscaler__clip': False, 'minmaxscaler__copy': True, 'minmaxscaler__feature_range': (0, 1), 'randomforestclassifier__bootstrap': True, 'randomforestclassifier__ccp_alpha': 0.0, 'randomforestclassifier__class_weight': None, 'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_leaf_nodes': None, 'randomforestclassifier__max_samples': None, 'randomforestclassifier__min_impurity_decrease': 0.0, 'randomforestclassifier__min_impurity_split': None, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__min_weight_fraction_leaf': 0.0, 'randomforestclassifier__n_estimators': 100, 'randomforestclassifier

In [195]:
hyperparameters = { 'randomforestclassifier__n_estimators' : [50, 100, 300, 500, 1000],
                    'randomforestclassifier__max_features' : ['auto', 'sqrt', 'log2'],
                   'randomforestclassifier__max_depth': [None, 5, 50, 70, 100, 200, 500, 1000]}

In [ ]:
from sklearn.model_selection import GridSearchCV

clf2 = GridSearchCV(pipeline,param_grid=hyperparameters, cv=10)

clf2.fit(X, y.values.ravel())

In [ ]:
print(clf2.best_params_)

In [ ]:
print(clf2.refit)

In [ ]:
from sklearn.metrics import accuracy_score

model_pred = clf2.predict(X_test)
accuracy_score(y_test, model_pred)

# random forest 3

In [331]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 200,
                            criterion = 'gini',
                            min_samples_split = 2,
                            n_jobs = -1,
                            verbose= 1,
                            min_samples_leaf = 1,
                            class_weight = 'balanced_subsample')

rfc.fit(X, y.values.ravel())

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   13.6s finished


RandomForestClassifier(class_weight='balanced_subsample', n_estimators=200,
                       n_jobs=-1, verbose=1)

In [332]:
from sklearn.metrics import accuracy_score

model_pred = rfc.predict(X_test)
accuracy_score(y_test, model_pred)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    0.2s finished


0.8412197686645636

# XGboost

In [90]:
import xgboost as xgb

param = {
    'eta': 0.1, 
    'max_depth': 40,  
    'objective': 'multi:softprob',  
    'num_class': 10,
    'eval_metric': 'merror'} 

steps = 20  # The number of training iterations

In [91]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
y['class'] = lbl.fit_transform(y['class'].astype(str))
y_test['class'] = lbl.fit_transform(y_test['class'].astype(str))


In [92]:
D_train = xgb.DMatrix(X, label=y)
D_test = xgb.DMatrix(X_test, label=y_test)

In [93]:
xgb_model = xgb.train(param, D_train, steps)

In [94]:
from sklearn.metrics import accuracy_score

model_pred = xgb_model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in model_pred])
accuracy_score(y_test, best_preds)

0.8333333333333334

# neural

In [172]:
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from tensorflow import keras

from scikeras.wrappers import KerasClassifier

In [173]:
X = X.astype(np.float32)
#y = y.astype(np.int64)

In [174]:
def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model


In [178]:
neural = KerasClassifier(
    get_model,
    loss="categorical_crossentropy",
    hidden_layer_dim=100,
    optimizer='adam', 
    metrics=['accuracy'],
    epochs=100, 
    batch_size=5, 
    verbose=1,
    validation_split=0.1
)

In [179]:
neural.fit(X, y)

Epoch 1/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.6163 - accuracy: 0.7915 - val_loss: 0.5461 - val_accuracy: 0.8112
Epoch 2/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.4425 - accuracy: 0.8464 - val_loss: 0.5211 - val_accuracy: 0.8215
Epoch 3/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.3327 - accuracy: 0.8839 - val_loss: 0.5189 - val_accuracy: 0.8329
Epoch 4/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.2209 - accuracy: 0.9235 - val_loss: 0.6073 - val_accuracy: 0.8355
Epoch 5/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.1211 - accuracy: 0.9585 - val_loss: 0.8133 - val_accuracy: 0.8318
Epoch 6/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.0644 - accuracy: 0.9778 - val_loss: 0.9858 - val_accuracy: 0.8263
Epoch 7/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.0369 - accuracy: 0.9879 - val_loss: 1.1693 -

Epoch 57/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.0060 - accuracy: 0.9989 - val_loss: 4.5473 - val_accuracy: 0.8254
Epoch 58/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.0046 - accuracy: 0.9991 - val_loss: 4.8163 - val_accuracy: 0.8252
Epoch 59/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.0063 - accuracy: 0.9987 - val_loss: 4.7732 - val_accuracy: 0.8243
Epoch 60/100
8216/8216 [==============================] - 14s 2ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 4.8534 - val_accuracy: 0.8245
Epoch 61/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 4.7938 - val_accuracy: 0.8291
Epoch 62/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.0042 - accuracy: 0.9993 - val_loss: 4.9458 - val_accuracy: 0.8239
Epoch 63/100
8216/8216 [==============================] - 15s 2ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 4

KerasClassifier(
	model=<function get_model at 0x7f98eef12dc0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=categorical_crossentropy
	metrics=['accuracy']
	batch_size=5
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.1
	shuffle=True
	run_eagerly=False
	epochs=100
	hidden_layer_dim=100
	class_weight=None
)

In [180]:
X_test = X_test.astype(np.float32)
#y_test = y_test.astype(np.int64)

In [181]:
from sklearn.metrics import accuracy_score

model_pred = neural.predict(X_test)
accuracy_score(y_test, model_pred)

2283/2283 [==============================] - 1s 633us/step


0.8307045215562566

# navie Bayes

In [31]:
from sklearn.naive_bayes import MultinomialNB

navie = MultinomialNB(alpha=1.0)#default value of 1.0

navie.fit(X, y.values.ravel())

MultinomialNB()

In [32]:
from sklearn.metrics import accuracy_score

model_pred = navie.predict(X_test)
accuracy_score(y_test, model_pred)

0.7129337539432177

# MLPClassifier

In [16]:
from sklearn.neural_network import MLPClassifier

neural = MLPClassifier(solver='adam', alpha=1e-6,
                    hidden_layer_sizes=(500, 200), random_state=1)

neural.fit(X, y.values.ravel())

MLPClassifier(alpha=1e-06, hidden_layer_sizes=(500, 200), random_state=1)

In [18]:
from sklearn.metrics import accuracy_score

model_pred = neural.predict(X_test)
accuracy_score(y_test, model_pred)

0.8415702769015072

# k neighbors

In [216]:
from sklearn.neighbors import KNeighborsClassifier

k = KNeighborsClassifier(n_neighbors=30)

k.fit(X, y.values.ravel())

KNeighborsClassifier(n_neighbors=30)

In [217]:
from sklearn.metrics import accuracy_score

model_pred = k.predict(X_test)
accuracy_score(y_test, model_pred)

0.7169645986680687

# logistic regression

In [219]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(random_state=0,solver='lbfgs', max_iter=1000)

log.fit(X, y.values.ravel())

LogisticRegression(max_iter=1000, random_state=0)

In [220]:
from sklearn.metrics import accuracy_score

model_pred = log.predict(X_test)
accuracy_score(y_test, model_pred)

0.8310550297932001

# logistic CV

In [229]:
# Train Logistic Regression Model
from sklearn.linear_model import LogisticRegressionCV

##choose model hyperparams
logCV = LogisticRegressionCV(cv=10,
                             random_state=0,
                             solver='lbfgs', 
                             max_iter=1000,
                            verbose = True,
                            n_jobs = -1)

logCV.fit(X, y.values.ravel())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 265.9min finished


LogisticRegressionCV(cv=10, max_iter=1000, n_jobs=-1, random_state=0,
                     verbose=True)

In [230]:
from sklearn.metrics import accuracy_score

model_pred = logCV.predict(X_test)
accuracy_score(y_test, model_pred)

0.832807570977918

# Gradient Boosting 

In [191]:
from sklearn.ensemble import GradientBoostingClassifier

graboost = GradientBoostingClassifier(n_estimators=100,
                                      learning_rate=1, 
                                      max_depth=2, 
                                      random_state=60,
                                     verbose=1)

graboost.fit(X, y.values.ravel())

      Iter       Train Loss   Remaining Time 
         1           1.4246            6.67m
         2           1.8312            6.59m
         3        2269.5261            6.53m
         4 27333551648544310066937856.0000            6.48m
         5 27333580240851755201462272.0000            6.42m
         6 33576182642863598135672832.0000            6.35m
         7 33576182642863598135672832.0000            6.27m
         8 33576182642878441542647808.0000            6.20m
         9 33576182642878441542647808.0000            6.14m
        10 33576182642878441542647808.0000            6.07m
        20 36652091169572429297090560.0000            5.39m
        30 54055747243785713912656361969339554880379053604157198743749700994763081926454606150868934498181907305887926550077112320.0000            4.71m
        40 54055747243785713912656361969339554880379053604157198743749700994763081926454606150868934498181907305887926550077112320.0000            4.04m
        50 5405574724378571391265

GradientBoostingClassifier(learning_rate=1, max_depth=2, random_state=60,
                           verbose=1)

In [192]:
from sklearn.metrics import accuracy_score

model_pred = graboost.predict(X_test)
accuracy_score(y_test, model_pred)

0.6714861549246407

# svm poly

In [3]:
from sklearn.svm import SVC
svm_poly = SVC(kernel='poly', degree=8,verbose=True)
svm_poly.fit(X, y.values.ravel())

[LibSVM]

SVC(degree=8, kernel='poly', verbose=True)

In [5]:
from sklearn.metrics import accuracy_score
model_pred = svm_poly.predict(X_test)
accuracy_score(y_test, model_pred)

0.5932351910269892

# Bagged Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier

bdt = BaggingClassifier(base_estimator=DecisionTreeClassifier(min_samples_leaf=3),
                           n_estimators=50, max_samples=100, bootstrap=True, random_state=0)

bdt.fit(X, y.values.ravel())

BaggingClassifier(base_estimator=DecisionTreeClassifier(min_samples_leaf=3),
                  max_samples=100, n_estimators=50, random_state=0)

In [13]:
from sklearn.metrics import accuracy_score
model_pred = bdt.predict(X_test)
accuracy_score(y_test, model_pred)

0.6158429723098493

# AdaBoost

In [14]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(n_estimators=30, 
                            base_estimator=DecisionTreeClassifier(max_depth=5), 
                            learning_rate=1e-5, random_state=0)

adaboost.fit(X, y.values.ravel())

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=1e-05, n_estimators=30, random_state=0)

In [15]:
from sklearn.metrics import accuracy_score
model_pred = adaboost.predict(X_test)
accuracy_score(y_test, model_pred)

0.37276550998948477

# catboost

In [ ]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(iterations=500,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='AUC',
                             #random_seed = RANDOM_STATE,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             #metric_period = VERBOSE_EVAL,
                             od_wait=100)

catboost.fit(X, y.values.ravel())

In [ ]:
from sklearn.metrics import accuracy_score
model_pred = catboost.predict(X_test)
accuracy_score(y_test, model_pred)

# prepare submit.csv

In [325]:
# read in sample sub
sample = pd.read_csv('./raw_data/sample.csv')
sample

,id,variety
0,57056,Syrah
1,57057,Bordeaux-style Red Blend
2,57058,Cabernet Sauvignon
3,57059,Cabernet Sauvignon
4,57060,Rosé
...,...,...
14261,71317,Bordeaux-style Red Blend
14262,71318,Sauvignon Blanc
14263,71319,Syrah
14264,71320,Chardonnay


In [326]:
# predict in test data set
ID = sample[['id']]

In [327]:
# choose which model to use and get prediction
#test = test.astype(np.float32)
Y_predicted = rfc.predict(test)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 300 out of 300 | elapsed:    0.4s finished


In [328]:
# transfer prediction to data frame
Y_predicted = pd.DataFrame(Y_predicted, columns=['variety'])
Y_predicted

,variety
0,Pinot Noir
1,Chardonnay
2,Merlot
3,Red Blend
4,Sauvignon Blanc
...,...
14261,Rosé
14262,Bordeaux-style Red Blend
14263,Cabernet Sauvignon
14264,Chardonnay


In [329]:
# combine id and prediction together
submit = pd.concat([ID,Y_predicted], axis=1, sort=False)
submit

,id,variety
0,57056,Pinot Noir
1,57057,Chardonnay
2,57058,Merlot
3,57059,Red Blend
4,57060,Sauvignon Blanc
...,...,...
14261,71317,Rosé
14262,71318,Bordeaux-style Red Blend
14263,71319,Cabernet Sauvignon
14264,71320,Chardonnay


In [330]:
# write the file
submit.to_csv("./submit.csv",index=False)